In [101]:
#A cross-platform GUI automation Python module for human beings 
#Used to programmatically control the mouse & keyboard
#!pip3 install pyautogui 
import pyautogui as pag
import pandas as pd
import numpy as np
from time import sleep
import csv
import json
from bs4 import BeautifulSoup 
import requests
import os
import re
import traceback

In [102]:
#!pip3 install selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

In [103]:
#Read restaurants' info file collected from Google API
with open('Kaohsiung.json' , 'r',encoding="utf-8") as reader:
    jf = json.loads(reader.read())

#Make sure we collect unique url link
unique ={each['url'] : each for each in jf} .values()

df = pd.DataFrame(unique)
df = df[['name','url','user_ratings_total']]

#If the number of reviews exceeds 928, we have to use another method to collect reviews
#Therefore, we only select the restaurant with less than 928 reviews
df = df.dropna()
df= df.loc[df['user_ratings_total'] < 928, ["name", "url", "user_ratings_total"]]
df

,name,url,user_ratings_total
0,ECO BURGER 愛客漢堡蔬食概念館,https://maps.google.com/?cid=17386114332409612828,427.0
1,葉家滷豬腳飯,https://maps.google.com/?cid=9353427274532494446,11.0
2,太陽蕃茄拉麵-左營新光店,https://maps.google.com/?cid=7164561221241711928,76.0
3,開動了 日本家庭料理,https://maps.google.com/?cid=5196209669921177601,258.0
4,I'm gogi,https://maps.google.com/?cid=2326288507511006167,565.0
...,...,...,...
3414,板產屋,https://maps.google.com/?cid=1654951593283734623,23.0
3415,美濃湖餐廳,https://maps.google.com/?cid=4683635518148065741,4.0
3416,早安美芝城旗山華中,https://maps.google.com/?cid=10863063057419433031,6.0
3417,品鄉小火鍋,https://maps.google.com/?cid=2284863645130067222,20.0


In [135]:
#Put the restaurant's name and URL in a list for further use
name = df["name"].tolist()
link = df["url"].tolist()

rests_name = []
url_link = []

#We need to separate the list because we use six computers to run this code
#Therefore, we use for loop to cut the list into several parts
for i in range(2256,3065):
    rests_name.append(name[i])
    url_link.append(link[i])

#Transform the list into data frame
oput_name = pd.DataFrame({'name':rests_name})
oput_link = pd.DataFrame({'url':url_link})
oput_rests = pd.concat([oput_name, oput_link], axis=1)
oput_rests

,name,url
0,肯德基-小港沿海店,https://maps.google.com/?cid=2264261495252445018
1,八方雲集（小港康莊店）,https://maps.google.com/?cid=1834428923719999011
2,潮州牛肉店,https://maps.google.com/?cid=13094790248021812374
3,上饌美食餐廳/四川菜/廣東菜/中餐/晚餐 /麻辣醬,https://maps.google.com/?cid=14132182261471067182
4,元味自助餐,https://maps.google.com/?cid=8969292891553465416
...,...,...
804,板產屋,https://maps.google.com/?cid=1654951593283734623
805,美濃湖餐廳,https://maps.google.com/?cid=4683635518148065741
806,早安美芝城旗山華中,https://maps.google.com/?cid=10863063057419433031
807,品鄉小火鍋,https://maps.google.com/?cid=2284863645130067222


In [131]:
# Start to scrap information on Google Maps!
def visit():
    driver.get(url)
    sleep(3) 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    #Collect the information about the restaurant on the first page
    try:
        title_element = soup.find('div', class_='section-hero-header-title-top-container')
        poiName_ele = title_element.find('h1', class_='section-hero-header-title-title GLOBAL__gm2-headline-5')
        name = poiName_ele.find('span').getText()
        name = name.replace('/','')
        rating_sections = title_element.find_all('div', class_='gm2-body-2')
        genre = rating_sections[1].find('button', class_='widget-pane-link').getText()
        totalReviews_section = title_element.find('span', class_='reviews-tap-area reviews-tap-area-enabled')

        totalNumReview_text = totalReviews_section.find_all('button')[0].getText()
#         print(totalNumReview_text)
        if "(" in totalNumReview_text and ")" in totalNumReview_text:
            totalNumReview = int(re.sub('[(),]', '', totalNumReview_text))
        else:
            totalNumReview = int(re.sub('[,]', '', totalNumReview_text.split(" ")[0]))
            
        
        listPost.append({'URL':url,
                         'Restaurant':name,
                         'Genre': genre,
                         'Total_reviews': totalNumReview,
                         'Review':[]
                        })   
            
    except:
        print(f'unable to collect poi information on {url}')
        traceback.print_exc()
     

    #Simulate mouse to scroll down
    pag.moveTo(200, 400)     
    pag.click()
    pag.scroll(-1000)
    sleep(1)



    #Click on "more reviews" button
    try:
        driver.find_element(By.CSS_SELECTOR,'button.allxGeDnJMl__button.allxGeDnJMl__button-text').click()
    except:
        try:
            div = driver.find_elements(By.CSS_SELECTOR,'div.section-review.ripple-container.GLOBAL__gm2-body-2')
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            stars=[]      
            names=[]

            for a in soup.select('span.section-review-stars'):   
                stars.append(a['aria-label'])     

            for b in soup.select('div.section-review-title'):
                names.append(b)

            for i, element in enumerate(soup.select('div.section-review.ripple-container.GLOBAL__gm2-body-2')):

                Content = element.find('span',{'class':'section-review-text'})

                listPost[len(listPost)-1]['Review'].append({'Index':i,
                                 'Author_name':names[i].text,
                                 'Stars':stars[i],
                                 'Time':element.find('span',{'class':'section-review-publish-date'}).text,
                                 'Content':Content.text})  
        except Exception:
            pass
   
    
    
def scroll():
    
    #Click on a blank part in the review section for further action.
    pag.moveTo(200, 400)     
    pag.click()
    
    #Keep scrolling down until there is no review
    i = 0
    while i < len(driver.find_elements(By.CSS_SELECTOR,'div.section-review.ripple-container.GLOBAL__gm2-body-2')):
        for j in range(5):
            pag.scroll(-1000000)
        sleep(1)
        i=i+1
       
    
def parse():

    #Collect all reviews
    try:

        div = driver.find_elements(By.CSS_SELECTOR,'div.section-review.ripple-container.GLOBAL__gm2-body-2')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        stars=[]      
        names=[]
        
        for a in soup.select('span.section-review-stars'):   
            stars.append(a['aria-label'])     
        
        for b in soup.select('div.section-review-title'):
            names.append(b)
        
        for i, element in enumerate(soup.select('div.section-review.ripple-container.GLOBAL__gm2-body-2')):
    
            Content = element.find('span',{'class':'section-review-text'})

            listPost[len(listPost)-1]['Review'].append({'Index':i,
                             'Author_name':names[i].text,
                             'Stars':stars[i],
                             'Time':element.find('span',{'class':'section-review-publish-date'}).text,
                             'Content':Content.text})         

    except TimeoutException:
        print('等待逾時！')


def end():    
    driver.quit()
    

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = options)  
    

if __name__ == '__main__':

    
    listPost = []
    
    for url in url_link:
        visit()
        scroll()
        parse()
        
        
    end()

    with open("google_review_Kaohsiung3_2.json", "w",encoding="utf-8") as f:
        json.dump(listPost, f, ensure_ascii=False, indent=4)

In [114]:
#If the code above shuts down without finishing the list, we can save the file here

with open("google_review_Kaohsiung3_2.json", "w",encoding="utf-8") as f:
    json.dump(listPost, f, ensure_ascii=False, indent=4)

In [121]:
#Test if the file saved successfully
with open('google_review_Kaohsiung3_2.json' , 'r',encoding="utf-8") as reader:
    jf = json.loads(reader.read())

oput_jf = pd.DataFrame(jf)
oput_jf

,URL,Restaurant,Genre,Total_reviews,Review
0,https://maps.google.com/?cid=10448493171967761502,台北永和豆漿,早餐餐廳,12,"[{'Index': 0, 'Author_name': ' 吳宣 ', 'Stars': ..."
1,https://maps.google.com/?cid=7228843344426488708,品福牛肉麵,餐廳,50,"[{'Index': 0, 'Author_name': ' 郭永財 ', 'Stars':..."
2,https://maps.google.com/?cid=14654354459455160746,鍾家 刈包飯糰,早餐餐廳,20,"[{'Index': 0, 'Author_name': ' 余潔 ', 'Stars': ..."
3,https://maps.google.com/?cid=4627327091185751453,早安美芝城鳳山八德,早餐餐廳,29,"[{'Index': 0, 'Author_name': ' Allen Allen ', ..."
4,https://maps.google.com/?cid=14235265232493220914,濰克早午餐-旗津店,早午餐餐廳,131,"[{'Index': 0, 'Author_name': ' Andy Tsai ', 'S..."
...,...,...,...,...,...
250,https://maps.google.com/?cid=353514311532642499,唯尼好,早午餐餐廳,290,"[{'Index': 0, 'Author_name': ' 蔡仲道 ', 'Stars':..."
251,https://maps.google.com/?cid=1407505706305109222,肉來了牛排館【鳳山店】,餐廳,13,"[{'Index': 0, 'Author_name': ' talmud ace ', '..."
252,https://maps.google.com/?cid=5572593252791006719,喵公爵,餐廳,13,"[{'Index': 0, 'Author_name': ' 重機熊貓人 ', 'Stars..."
253,https://maps.google.com/?cid=3469919614200450259,Cijin Sunset Bar 旗津沙灘吧,餐廳,464,"[{'Index': 0, 'Author_name': ' CC Lin ', 'Star..."
